In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import sklearn
from matplotlib import pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense, Dropout
from sklearn.preprocessing import MinMaxScaler
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import GridSearchCV
import tensorflow as tf
from sklearn.metrics import make_scorer



In [2]:
def search(data):
    # 时间输入
    print('请分别输入时间')
    print('请输入日：')
    day = input()
    day = int(day)
    print('请输入时：')
    hour = input()
    hour = int(hour)
    print('请输入分：')
    minute = input()
    minute = int(minute)
    # 将时间转换为时间戳类型
    time = day * 60 * 24 + hour * 60 + minute
    Time = pd.to_datetime([time], unit='m', origin=pd.Timestamp('2022-05-31'))
    # 记录索引
    index = 0
    for x in data['Datetime']:
        if x == Time:
            print('成功')
            break
        index += 1
    # 打印日期查询得到的power值
    print('Datetime:', data['Datetime'][index])
    print('Power(MW):', data['Power(MW)'][index])

    return

def visualize(data):
    print("请输入所要查看数据的日期")
    print("请输入日:")
    day = input()

    pre='2022-6-'
    day1= pre+day

    day= str(int (day)+1)
    day2=pre+day
    #筛选日期
    data = data[(data['Datetime'] >= day1) & (data['Datetime'] < day2)]

    #横坐标数据
    xtime = data['Datetime']
    #纵坐标数据
    ypower = data['Power(MW)']

    #横纵坐标数据导入图表
    plt.plot(xtime, ypower)
    #横坐标
    plt.xlabel('Datetime', fontsize=16)
    #纵坐标
    plt.ylabel('Power(MW)', fontsize=16)
    plt.show()

    return


def test(data):
    data = data[(data['Datetime'] >= '2022-6-13') & (data['Datetime'] < '2022-6-14')]
    print(type('2022-6-13'))
    return


In [ ]:
i=1
while i==1:
    if __name__ == '__main__':
        print("1.风力发电")
        print("2.光伏发电")
        print("请选择想要展示的数据")
        num_1=int(input())
        print('请输入history_nmp')
        add_1=input('请输入history_nmp')
        print('请输入history_power')
        add_2=input('请输入history_power')
        print('thanks')
        # execl文件路径
        if num_1==1:
            print('please wait')
            
            # num_1=1
            # 预处理输入数据
            def preprocessing(nmp_df,power_df):
                print('Date of his_weather:',nmp_df.head(10))
                print('Date of his_power:',power_df.head(10))
                fix_df=pd.merge(nmp_df, power_df, on=['Datetime'], how='left')
                print(fix_df.head(10))
                print(fix_df.describe())
                return fix_df
            # 读取数据
            his_nmp_fd_df = pd.read_csv('.\\data\\FD001\\03 His_NWP_FD.csv',parse_dates=["Datetime"],index_col=[0])
            his_power_fd_df = pd.read_csv('.\\data\\FD001\\02_His_Power_FD.csv',parse_dates=["Datetime"],index_col=[0])
            his_fd_df=preprocessing(his_nmp_fd_df,his_power_fd_df)
            his_fd_df.to_csv('his_df.csv',encoding = 'utf-8') #存储合并后数据
            df=pd.read_csv('his_df.csv',parse_dates=["Datetime"],index_col=[0])
            # 划分train valid test集
            fd_df_train = df[:int(0.6*len(df))]
            fd_df_valid = df[int(0.6*len(df)):int(0.8*len(df))]
            fd_df_test =df[int(0.8*len(df)):]
            # 对数据进行归一化处理
            scaler = MinMaxScaler(feature_range=(0,1))
            scaler = scaler.fit(df)
            df_for_training_scaled = scaler.transform(fd_df_train)
            df_for_validing_scaled=scaler.transform(fd_df_valid)
            df_for_testing_scaled=scaler.transform(fd_df_test)
            df_scaled=scaler.transform(df)
            # 分离x与y
            def createXY(array, days_used_to_train=7):
                features = list()
                labels = list()

                for i in range(days_used_to_train, len(array)):
                    features.append(array[i-days_used_to_train:i, :])
                    labels.append(array[i, -1])
                return np.array(features), np.array(labels)

            train_X,train_y=createXY(df_for_training_scaled)
            valid_X,valid_y=createXY(df_for_validing_scaled)
            test_X,test_y=createXY(df_for_testing_scaled)
            df_scaled_X,df_scaled_y=createXY(df_scaled)
            # 看一下历史波形
            plt.plot(range(len(df['Power(MW)'])), df['Power(MW)'], label='history')
            # 定义一个新的评价函数
            def ev(y_true, y_predict):
                difference=((np.array(y_true) - np.array(y_predict.flatten()))**2).sum()
                return float(difference) 
                from sklearn.metrics import make_scorer
            my_scorer = make_scorer(ev, greater_is_better=False) 
            scoring = {
                'ev': my_scorer
            }
            # 建立模型函数
            def build_model(size):
                tf.random.set_seed(1) #设置随机数种子
                # Create grid_model
                grid_model = tf.keras.Sequential()
                grid_model.add(tf.keras.layers.LSTM(units=64))
                grid_model.add(tf.keras.layers.Dense(64))
                grid_model.add(tf.keras.layers.Dense(size))
                grid_model.add(tf.keras.layers.Dense(1))

                # Compile the grid_model
                grid_model.compile(
                optimizer='adam',
                loss='mse')

                return grid_model
                from keras.wrappers.scikit_learn import KerasClassifier
            grid_model = KerasRegressor(build_fn=build_model)
            parameters = {'batch_size' : [16],'epochs' : [40],'size' : [32]}
            grid_search = GridSearchCV(estimator = grid_model,
                                      param_grid = parameters,
                                      scoring=scoring, 
                                       refit ='ev', 
                                       n_jobs=1,
                                       cv = 2)
            grid_search = grid_search.fit(train_X, train_y,validation_data=(valid_X, valid_y))
            # 输出最佳模型为my_model
            my_model=grid_search.best_estimator_.model
             # 预测未来
            # 从past数据中抽取7组作为未来数据的开头
            df_30_days_past=df.iloc[-7:,:]
            # 提取未来数据
            df_30_days_future=pd.read_csv('.\\data\\FD001\\04 Fore_NWP_FD.csv',parse_dates=["Datetime"],index_col=[0])
            df_30_days_future.shape
            df_30_days_future["Power(MW)"]=0 # 为方便后续操作，为未来数据添加一power列
            # 归一化过去数据
            old_scaled_array=scaler.transform(df_30_days_past)
            # 归一化未来数据
            new_scaled_array=scaler.transform(df_30_days_future)
            new_scaled_df=pd.DataFrame(new_scaled_array)
            new_scaled_df.iloc[:,-1]=np.nan #将未来数据表中power转为NaN
            full_df=pd.concat([pd.DataFrame(old_scaled_array),new_scaled_df]).reset_index().drop(["index"],axis=1) #合并表格
            # 开始逐点预测
            full_df_scaled_array=full_df.values
            all_data=[]
            time_step=7
            for i in range(time_step,len(full_df_scaled_array)):
                  data_x=[]
                  data_x.append( full_df_scaled_array[i-time_step :i , 0:full_df_scaled_array.shape[1]])
                  data_x=np.array(data_x)
                  prediction=my_model.predict(data_x)
                  if prediction<0:
                        prediction=0
                  all_data.append(prediction)
                  full_df.iloc[i,-1]=prediction
            all_data=np.array(all_data).flatten()
            # 反归一化操作
            df_fin=pd.read_csv('.\\data\\FD001\\04 Fore_NWP_FD.csv',parse_dates=["Datetime"],index_col=[0])
            len(df_fin)
            df_fin['Power(MW)']=0
            df_fin['Power(MW)']=all_data
            df_fin
            predicted_data = scaler.inverse_transform(df_fin)
            fd_time=pd.read_csv('.\\data\\YC_Result_FD.csv')
            fd_time['Power(MW)']=predicted_data[:,-1]
            fd_time.to_csv('feature_fd_power.csv',index=0)
            fd_time.to_excel('feature_fd_power.xlsx',index=0)
            print('success1')
            fpath = '.\\feature_fd_power.xlsx'
            #绘制反归一化后整体发电量图
            retotal = np.hstack((df['Power(MW)'],predicted_data[:,-1]))
            plt.plot(range(len(df['Power(MW)'])), df['Power(MW)'], label='Prediction')
            plt.plot(range(len(retotal)), retotal, label='Prediction',color='coral')
            plt.plot(range(len(df['Power(MW)'])), df['Power(MW)'], label='Prediction')

        if num_1==2:
            print('please wait2')
            #光伏代码放这
            
            
            
            
            
            #光伏end
            print('success2')
            fpath = '.\\feature_fd_power.xlsx'
        # 获取execl文件数据
        data = pd.read_excel(fpath)
        # 将数据中的时间对象转换为时间戳
        data['Datetime'] = pd.to_datetime(data['Datetime'])
        print('1.查询')
        print('2.可视化')
        print('请选择要进行的操作：')
        num=input()
        if num=='1':
            # 日期查询
            search(data)
        elif num=='2':
            if num_1==1:
                # 可视化
                plt.plot(range(len(df['Power(MW)'])), df['Power(MW)'], label='Prediction')
                plt.plot(range(len(retotal)), retotal, label='Prediction',color='coral') 
                plt.plot(range(len(df['Power(MW)'])), df['Power(MW)'], label='Prediction')
                plt.show()
            if num_1==2:
                plt.plot(range(len(all_data_fea_Denormalization)), np.array(all_data_fea_Denormalization).flatten(), label='Prediction')
            visualize(data)
    i=int(input("Restart? 0 or 1  "))


1.风力发电
2.光伏发电
请选择想要展示的数据
1
请输入history_nmp
请输入history_nmp1
请输入history_power
请输入history_power1
thanks
please wait
Date of his_weather:                      Speed10  Direction10  Speed30  Direction30  Speed50  \
Datetime                                                                   
2022-07-15 00:00:00   3.3696     290.3172   4.1829     295.0334   4.9253   
2022-07-15 00:15:00   3.3092     291.4477   4.1100     296.1286   4.8389   
2022-07-15 00:30:00   3.4671     295.0868   4.2669     299.3285   5.0000   
2022-07-15 00:45:00   3.6896     296.9123   4.4933     300.7888   5.2253   
2022-07-15 01:00:00   3.7095     298.4995   4.5220     302.3546   5.2791   
2022-07-15 01:15:00   3.5143     302.9213   4.3541     306.5014   5.1350   
2022-07-15 01:30:00   3.3300     306.6978   4.1933     310.2604   4.9758   
2022-07-15 01:45:00   3.0254     311.3819   3.9033     314.7924   4.7046   
2022-07-15 02:00:00   3.7342     313.9150   4.4407     315.3649   5.0855   
2022-07-15 02:15:00   3.1972   

Epoch 1/40


C:\Users\23966\AppData\Local\Temp\ipykernel_6124\3377300610.py:85: DeprecationWarning: KerasRegressor is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  grid_model = KerasRegressor(build_fn=build_model)


13/13 [==============================] - 2s 38ms/step - loss: 0.1107 - val_loss: 0.0201
Epoch 2/40
13/13 [==============================] - 0s 6ms/step - loss: 0.0319 - val_loss: 0.0876
Epoch 3/40
13/13 [==============================] - 0s 6ms/step - loss: 0.0165 - val_loss: 0.0147
Epoch 4/40
13/13 [==============================] - 0s 6ms/step - loss: 0.0127 - val_loss: 0.0242
Epoch 5/40
13/13 [==============================] - 0s 6ms/step - loss: 0.0109 - val_loss: 0.0121
Epoch 6/40
13/13 [==============================] - 0s 6ms/step - loss: 0.0090 - val_loss: 0.0054
Epoch 7/40
13/13 [==============================] - 0s 6ms/step - loss: 0.0076 - val_loss: 0.0021
Epoch 8/40
13/13 [==============================] - 0s 6ms/step - loss: 0.0079 - val_loss: 0.0054
Epoch 9/40
13/13 [==============================] - 0s 6ms/step - loss: 0.0073 - val_loss: 0.0023
Epoch 10/40
13/13 [==============================] - 0s 6ms/step - loss: 0.0074 - val_loss: 0.0018
Epoch 11/40
13/13 [==========

Epoch 3/40
25/25 [==============================] - 0s 4ms/step - loss: 0.0116 - val_loss: 0.0032
Epoch 4/40
25/25 [==============================] - 0s 4ms/step - loss: 0.0102 - val_loss: 0.0057
Epoch 5/40
25/25 [==============================] - 0s 4ms/step - loss: 0.0102 - val_loss: 0.0131
Epoch 6/40
25/25 [==============================] - 0s 4ms/step - loss: 0.0096 - val_loss: 0.0022
Epoch 7/40
25/25 [==============================] - 0s 4ms/step - loss: 0.0094 - val_loss: 0.0038
Epoch 8/40
25/25 [==============================] - 0s 4ms/step - loss: 0.0090 - val_loss: 0.0013
Epoch 9/40
25/25 [==============================] - 0s 4ms/step - loss: 0.0084 - val_loss: 0.0051
Epoch 10/40
25/25 [==============================] - 0s 4ms/step - loss: 0.0080 - val_loss: 0.0048
Epoch 11/40
25/25 [==============================] - 0s 4ms/step - loss: 0.0080 - val_loss: 0.0037
Epoch 12/40
25/25 [==============================] - 0s 4ms/step - loss: 0.0083 - val_loss: 0.0030
Epoch 13/40
25/25

...

1/1 [==============================] - 0s 14ms/step


1/1 [==============================] - 0s 14ms/step


1/1 [==============================] - 0s 15ms/step


C:\Users\23966\AppData\Local\Temp\ipykernel_6124\3377300610.py:123: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  all_data=np.array(all_data).flatten()


success1
1.查询
2.可视化
请选择要进行的操作：


In [ ]:
plt.plot(range(len(df['Power(MW)'])), df['Power(MW)'], label='Prediction')
plt.plot(range(len(retotal)), retotal, label='Prediction',color='coral') 
plt.plot(range(len(df['Power(MW)'])), df['Power(MW)'], label='Prediction')
plt.show()